In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/bert-initialization/kaggle/working/bert_base_uncased_pretrained_model_pytorch_primary/config.json
/kaggle/input/bert-initialization/kaggle/working/bert_base_uncased_pretrained_model_pytorch_primary/tokenizer.json
/kaggle/input/bert-initialization/kaggle/working/bert_base_uncased_pretrained_model_pytorch_primary/tokenizer_config.json
/kaggle/input/bert-initialization/kaggle/working/bert_base_uncased_pretrained_model_pytorch_primary/model.safetensors
/kaggle/input/bert-initialization/kaggle/working/bert_base_uncased_pretrained_model_pytorch_primary/special_tokens_map.json
/kaggle/input/bert-initialization/kaggle/working/bert_base_uncased_pretrained_model_pytorch_primary/vocab.txt
/kaggle/input/distilbert-initialization/kaggle/working

In [2]:
# !pip install -U git+https://github.com/huggingface/transformers.git
# !pip install -U git+https://github.com/huggingface/accelerate.git

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
train_prompts = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')
train_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")

In [5]:
# model_name = "distilbert-base-uncased-finetuned-sst-2-english"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [6]:
# save_path = '/kaggle/working/distilbert_base_uncased_finetuned_model'

# !mkdir {save_path}

# model.save_pretrained(save_path)
# tokenizer.save_pretrained(save_path)

# !ls {save_path}
# !zip -r distilbert_uncased.zip {save_path}

In [7]:
load_path = "/kaggle/input/distilbert-initialization/kaggle/working/distilbert_base_uncased_finetuned_model"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(load_path)
model = AutoModelForSequenceClassification.from_pretrained(load_path, num_labels=2)

In [9]:
model.to("cuda")

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [10]:
tokenized_inputs = tokenizer(list(train_essays['text']), padding=True, truncation=True, 
                             return_tensors="pt")
labels = torch.tensor(train_essays['generated'])

dataset = TensorDataset(tokenized_inputs["input_ids"], 
                        tokenized_inputs["attention_mask"], labels)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [11]:
num_epochs = 50

model.train()

optimizer = AdamW(model.parameters(), lr=1e-5)

pbar1 = tqdm(total = len(dataloader))

for epoch in tqdm(range(num_epochs)):
    for i,batch in enumerate(dataloader):
        input_ids, attention_mask, label = batch
        input_ids, attention_mask, label = input_ids.to("cuda"), attention_mask.to("cuda"), label.to("cuda") 

        outputs = model(input_ids, attention_mask=attention_mask, labels=label)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i%25==0:
            pbar1.update(i)

model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/173 [00:00<?, ?it/s]
525it [00:33, 20.85it/s]
1050it [01:09, 19.94it/s]
1575it [01:46, 19.86it/s]
2100it [02:23, 19.85it/s]
2625it [03:00, 19.86it/s]
3150it [03:37, 19.85it/s]
3675it [04:14, 19.86it/s]
4200it [04:51, 19.86it/s]
4725it [05:28, 19.85it/s]
5250it [06:05, 19.86it/s]
5775it [06:41, 19.86it/s]
6300it [07:18, 19.86it/s]
6825it [07:55, 19.86it/s]
7350it [08:32, 19.86it/s]
7875it [09:09, 19.86it/s]
8400it [09:46, 19.85it/s]
8925it [10:23, 19.86it/s]
9450it [11:00, 19.86it/s]
9975it [11:36, 19.86it/s]
10500it [12:13, 19.86it/s]
11025it [12:50, 19.85it/s]
11550it [13:27, 19.85it/s]
12075it [14:04, 19.85it/s]
12600it [14:41, 19.85it/s]
13125it [15:18, 19.85it

('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/vocab.txt',
 'fine_tuned_model/added_tokens.json',
 'fine_tuned_model/tokenizer.json')

In [12]:
test_essays = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

In [13]:
from tqdm import tqdm

predictions = []
with torch.no_grad():
    for text in train_essays.text:
        
        input_ids = tokenizer(text, 
                      padding=True, 
                      truncation=True,  
                      return_tensors='pt')["input_ids"].to("cuda")
        
        outputs = model(input_ids)
        predictions.append(torch.nn.functional.sigmoid(outputs.logits))

26250it [31:00, 19.86it/s]

In [14]:
preds = [np.array(a.to("cpu"))[0][1] for a in predictions]

In [15]:
print("ROC-AUC score train = ", roc_auc_score(train_essays['generated'],preds))

ROC-AUC score train =  1.0


In [16]:
predictions_test = []
with torch.no_grad():
    for text in test_essays.text:
        
        input_ids = tokenizer(text, 
                      padding=True, 
                      truncation=True,  
                      return_tensors='pt')["input_ids"].to("cuda")
        
        outputs = model(input_ids)
        predictions_test.append(torch.nn.functional.sigmoid(outputs.logits))

In [17]:
preds_test = [np.array(a.to("cpu"))[0][1] for a in predictions_test]

In [18]:
test_essays['generated'] = preds_test

In [19]:
test_essays[['id', 'generated']].to_csv("submission.csv", index = False)